In [6]:
import datetime as dt
import time
import logging as log
from multiprocessing import Pool, TimeoutError

%run BreakSectionManager.ipynb
%run Util.ipynb
%run Constant.ipynb
%run Metric.ipynb
%run Operation.ipynb
%run Account.ipynb
%run Policy_Loader.ipynb
%run Evaluation.ipynb


class RocketRegression:
    
    def __init__(self, policy_name):
        self.policy_name = policy_name
        self.policy_loader = Policy_Loader(self.policy_name)
        self.debug = self.policy_loader.enabled_log_debug()
        self.context = self.policy_loader.get_context()
        
        self.level = self.context['level']
        self.short = self.context['short']
        self.mid = self.context['mid']
        
    def build_operation_file_path(self,symbol,regression_end_date):
        return report_dir+'operations_'+symbol+'_'+str(self.short)+'_'+str(self.mid)+'_'+self.level+'_'+regression_end_date+'_'+operation_version+'.csv'
    
    def build_deal_file_path(self,symbol,regression_end_date):
        return report_dir+'deals_'+symbol+'_'+str(self.short)+'_'+str(self.mid)+'_'+self.level+'_'+regression_end_date+'_'+operation_version+'.csv'
        
    def build_asset_file_path(self,symbol,regression_end_date):
        return report_dir+'account_asset_'+symbol+'_'+str(self.short)+'_'+str(self.mid)+'_'+self.level+'_'+regression_end_date+'_'+operation_version+'.csv'
    
    def persist(self,account,operation_df,symbol,regression_end_date):
        account.get_asset_df().to_csv(self.build_asset_file_path(symbol,regression_end_date),index=False)
        account.deal_df.to_csv(self.build_deal_file_path(symbol,regression_end_date),index=False)
        operation_df.to_csv(self.build_operation_file_path(symbol,regression_end_date), index=False)
        
    
    def start_regression_on_one_stock(self,symbol,regression_end_date):
        start = time.time()
        sectionManager = BreakSectionManager(symbol,self.context)
        section_df = sectionManager.get_all_break_sections()
        section_df = section_df[section_df['d_s_datetime']<to_datetime(regression_end_date)]
        
        metric = Metric(symbol,self.context)
        
        stock_start_date = metric.get_stock_start_date()
        stock_end_date = metric.get_stock_end_date()
        log.info('stock_start_date ' + stock_start_date+' stock_end_date:'+stock_end_date)
        
        account = Account(stock_start_date,stock_end_date,self.context,self.policy_name)
        operation = Operation(account,symbol,self.context,self.policy_name)
        
        for key,row in section_df.iterrows():
            start_date = row['d_s_date']
            end_date = row['d_e_date']
            
            start_datetime = to_datetime(start_date)
            end_datetime = to_datetime(end_date)

            buy_price = metric.get_cur_price(start_date)
            
            if(self.debug==True):
                print('\n'+row['symbol']+' section '+start_date+'===>'+end_date)
            while(start_datetime<=end_datetime):
                cur_date = start_datetime.strftime(YMD_format)
                
                if(metric.is_today_open(cur_date)):
                    account.daily_audit(cur_date)

                    if(account.can_open_new_stock()==True):
                        if(operation.is_buy_point(cur_date)==True):
                            operation.open_opsition(cur_date)

                    if(account.has_shares()==True):
                        if(operation.is_sell_point(cur_date)==True):
                            operation.sell_stock(cur_date)

                start_datetime += dt.timedelta(days = 1)
                
        operation_df = operation.get_operations()
        self.persist(account,operation_df,symbol,regression_end_date)
        
        evaluation = Evaluation(account,self.context,self.policy_name)
        evaluation_df = evaluation.get_evaluation_report(symbol,stock_start_date,stock_end_date,
                                                         operation_df)

        end = time.time()
        if(self.debug==False):
            print(symbol+' regression cost time ' + str(round(end-start,1))+' seconds')
        return evaluation_df
    
    
    def start_sync_regression(self,scale,regression_end_date):
        evaluation_df = pd.DataFrame(columns=evaluation_columns)
        symbols = get_symbols(scale,home)
        total = len(symbols) 
        
        for symbol in symbols:
            one_evaluation = self.start_regression_on_one_stock(symbol,regression_end_date)
            evaluation_df = pd.concat([one_evaluation,evaluation_df], ignore_index=True, sort=False)
           
        
        evaluation_df['loss_times'] =evaluation_df['loss_times'].astype('int')
        evaluation_df['deal_count'] =evaluation_df['deal_count'].astype('int')
        evaluation_df['r2'] =evaluation_df['r2'].astype('float')
        evaluation_df['rar'] =evaluation_df['rar'].astype('float')

        return evaluation_df
    
    def start_regression(self,scale,regression_end_date,process_num):
        print('context:' + str(self.context))
        
        if(process_num==1):
            return self.start_sync_regression(scale,regression_end_date)
        
        evaluation_df = pd.DataFrame(columns=evaluation_columns)
        symbols = get_symbols(scale,home)
        total = len(symbols) 
        
        pool = Pool(processes=process_num)
        tasks = []
        for symbol in symbols:
            task = pool.apply_async(self.start_regression_on_one_stock, (symbol,regression_end_date))
            tasks.append(task)
            
        for task in tasks:
            try:
                one_evaluation = task.get()
                evaluation_df = pd.concat([one_evaluation,evaluation_df], ignore_index=True, sort=False)
            except Exception as e:
                print(str(e))
        
        pool.close()
        pool.join()
        
        evaluation_df['loss_times'] =evaluation_df['loss_times'].astype('int')
        evaluation_df['deal_count'] =evaluation_df['deal_count'].astype('int')
        evaluation_df['r2'] =evaluation_df['r2'].astype('float')
        evaluation_df['rar'] =evaluation_df['rar'].astype('float')

        return evaluation_df
    

In [ ]:
regression = RocketRegression('rocket_break_v4_enable_reducing_position_based_ATR.json')

regression_end='2019-05-30'
scale=300
process_num=24

evaluation_df = regression.start_regression(scale,regression_end,process_num)
evaluation_df.describe()

context:{'short': 20, 'mid': 240, 'scale_start': 0, 'scale_end': 3, 'level': 'Day'}


/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


SZ#002345 regression cost time 33.5 seconds
SZ#002343 regression cost time 33.5 seconds


/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


SZ#002505 regression cost time 35.6 seconds


...

list.remove(x): x not in list
SZ#002517 regression cost time 15.2 seconds


/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


SZ#002194 regression cost time 22.4 seconds
SZ#002357 regression cost time 14.2 seconds
SZ#000014 regression cost time 51.8 seconds
SZ#000017 regression cost time 24.1 seconds
SZ#002518 regression cost time 19.0 seconds


/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


SZ#002519 regression cost time 14.4 seconds
SZ#002195 regression cost time 21.6 seconds
SZ#000820 regression cost time 47.7 seconds


In [ ]:
print('Helo')